# **Amazon Lookout for Equipment**
*Part 2 - Dataset creation*

### Notebook configuration update
Let's make sure that we have access to the latest version of the AWS Python packages. If you see a `pip` dependency error, check that the `boto3` version is ok: if it's greater than 1.17.48 (the first version that includes the `lookoutequipment` API), you can discard this error and move forward with the next cell:

In [ ]:
!pip install --quiet sagemaker==2.46.1 tqdm==4.62.1

import boto3
print(f'boto3 version: {boto3.__version__} (should be >= 1.17.48 to include Lookout for Equipment API)')

# Restart the current notebook to ensure we take into account the previous updates:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
boto3 version: 1.18.28 (should be >= 1.17.48 to include Lookout for Equipment API)


### Imports

In [1]:
import config
import os
import pandas as pd
import sagemaker
import sys
import time
import boto3
from datetime import datetime

# Helper functions for managing Lookout for Equipment API calls:
sys.path.append('../utils')
import lookout_equipment_utils as lookout

In [2]:
ROLE_ARN       = sagemaker.get_execution_role()
REGION_NAME    = boto3.session.Session().region_name
ASSET_ID       = config.ASSET_ID
DATASET_NAME   = config.DATASET_NAME
BUCKET         = config.BUCKET
PREFIX         = config.PREFIX_TRAINING

PROCESSED_DATA = os.path.join('..', 'data', 'processed', ASSET_ID)
TRAIN_DATA     = os.path.join(PROCESSED_DATA, 'training-data')

In [3]:
# List of the directories from the training data 
# directory: each directory corresponds to a subsystem:
components = []
for root, dirs, files in os.walk(f'{TRAIN_DATA}'):
    for subsystem in dirs:
        if subsystem != '.ipynb_checkpoints':
            components.append(subsystem)
        
components

['centrifugal-pump']

## Create a dataset
---

### Create data schema

First we need to setup the schema of your dataset. In the cell below, we define `DATASET_COMPONENT_FIELDS_MAP`. `DATASET_COMPONENT_FIELDS_MAP` is a Python dictonary (hashmap). The key of each entry in the dictionary is the `Component` name, and the value of each entry is a list of column names. The column names must exactly match the header in your CSV files. The order of the column names also need to exactly match:

```json
DATASET_COMPONENT_FIELDS_MAP = {
    "Component1": ['Timestamp', 'Tag1', 'Tag2',...],
    "Component2": ['Timestamp', 'Tag1', 'Tag2',...]
    ...
    "ComponentN": ['Timestamp', 'Tag1', 'Tag2',...]
}
```

We also need to make sure the component name **matches exactly** the name of the folder in S3 (everything is **case sensitive**). As an example, when creating the data schema for the example we are using here, we will build a the dictionary that will look like this:
```json
DATASET_COMPONENT_FIELDS_MAP = {
    "centrifugal-pump": ['Timestamp', 'Sensor0', 'Sensor1',... , 'Sensor29']
}
```
The following cell builds this map, then convert it into a JSON schema that follows the following format, which is ready to be processed by Lookout for Equipment:

```json
{
  "Components": [
    {
      "ComponentName": "centrifugal-pump",
      "Columns": [
        {"Name": "Timestamp", "Type": "DATETIME"},
        {"Name": "Sensor0", "Type": "DOUBLE"},
        {"Name": "Sensor1", "Type": "DOUBLE"},
        {"Name": "Sensor2", "Type": "DOUBLE"},
        {"Name": "Sensor3", "Type": "DOUBLE"},
          
        ...
          
        {"Name": "Sensor29", "Type": "DOUBLE"}
      ]
    }
  ]
}
```

In [4]:
DATASET_COMPONENT_FIELDS_MAP = dict()
for subsystem in components:
    subsystem_tags = ['Timestamp']
    for root, _, files in os.walk(f'{TRAIN_DATA}/{subsystem}'):
        for file in files:
            fname = os.path.join(root, file)
            current_subsystem_df = pd.read_csv(fname, nrows=1)
            subsystem_tags = subsystem_tags + current_subsystem_df.columns.tolist()[1:]

        DATASET_COMPONENT_FIELDS_MAP.update({subsystem: subsystem_tags})
        
        
lookout_dataset = lookout.LookoutEquipmentDataset(
    dataset_name=DATASET_NAME,
    component_fields_map=DATASET_COMPONENT_FIELDS_MAP,
    region_name=REGION_NAME,
    access_role_arn=ROLE_ARN
)

If you wanted to use the console, the following string would be the one to use to configure the **dataset schema**:

![Dataset creation with schema](assets/dataset-schema.png)

### Create the dataset
The following method encapsulate the [**CreateDataset**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_CreateDataset.html) API:

```python
lookout_client.create_dataset(
    DatasetName=self.dataset_name,
    DatasetSchema={
        'InlineDataSchema': "schema"
    }
)
```

In [5]:
lookout_dataset.create()

Dataset "27b53ba8-baa0-4298-809a-0afe9384c280" does not exist, creating it...



{'DatasetName': '27b53ba8-baa0-4298-809a-0afe9384c280',
 'DatasetArn': 'arn:aws:lookoutequipment:us-east-1:593512547852:dataset/27b53ba8-baa0-4298-809a-0afe9384c280/3359ff20-cc69-484d-902a-cc09577d6eff',
 'Status': 'CREATED',
 'ResponseMetadata': {'RequestId': '55af057a-dacb-459b-a877-b75ff340e5c2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '55af057a-dacb-459b-a877-b75ff340e5c2',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '218',
   'date': 'Fri, 03 Sep 2021 02:42:50 GMT'},
  'RetryAttempts': 0}}

The dataset is now created, but it is empty and ready to receive some timeseries data that we will ingest from the S3 location prepared in the previous notebook:

![Dataset created](assets/dataset-created.png)

## Ingest data into a dataset
---
Let's double check the values of all the parameters that will be used to ingest some data into an existing Lookout for Equipment dataset:

In [6]:
ROLE_ARN, BUCKET, PREFIX, DATASET_NAME

('arn:aws:iam::593512547852:role/LookoutForEquipmentSitewisePipeli-SageMakerIamRole-1MJQSDGDJXJTP',
 'l4esitewisejjhj-asset1-train-inference',
 '27b53ba8-baa0-4298-809a-0afe9384c280/training-data/',
 '27b53ba8-baa0-4298-809a-0afe9384c280')

Launch the ingestion job in the Lookout for Equipment dataset: the following method encapsulates the [**StartDataIngestionJob**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_StartDataIngestionJob.html) API:

```python
lookout_client.start_data_ingestion_job(
    DatasetName=DATASET_NAME,
    RoleArn=ROLE_ARN, 
    IngestionInputConfiguration={ 
        'S3InputConfiguration': { 
            'Bucket': BUCKET,
            'Prefix': PREFIX
        }
    }
)
```

In [7]:
response = lookout_dataset.ingest_data(BUCKET, PREFIX)

The ingestion is launched. With this amount of data (around 50 MB), it should take between less than 5 minutes:

![dataset_schema](assets/dataset-ingestion-in-progress.png)

We use the following cell to monitor the ingestion process by calling the [**DescribeDataIngestionJob**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_DescribeDataIngestionJob.html) API every 60 seconds:

In [8]:
# Get the ingestion job ID and status:
data_ingestion_job_id = response['JobId']
data_ingestion_status = response['Status']

# Wait until ingestion completes:
print("=====Polling Data Ingestion Status=====\n")
lookout_client = lookout.get_client(region_name=REGION_NAME)
print(str(pd.to_datetime(datetime.now()))[:19], "|", data_ingestion_status)

while data_ingestion_status == 'IN_PROGRESS':
    time.sleep(60)
    describe_data_ingestion_job_response = lookout_client.describe_data_ingestion_job(JobId=data_ingestion_job_id)
    data_ingestion_status = describe_data_ingestion_job_response['Status']
    print(str(pd.to_datetime(datetime.now()))[:19], "|", data_ingestion_status)
    
print("\n=====End of Polling Data Ingestion Status=====")

=====Polling Data Ingestion Status=====

2021-09-03 02:42:56 | IN_PROGRESS
2021-09-03 02:43:56 | IN_PROGRESS
2021-09-03 02:44:56 | IN_PROGRESS
2021-09-03 02:45:56 | IN_PROGRESS
2021-09-03 02:46:56 | IN_PROGRESS
2021-09-03 02:47:56 | SUCCESS

=====End of Polling Data Ingestion Status=====


In case any issue arise, you can inspect the API response available as a JSON document:

In [9]:
lookout_client.describe_data_ingestion_job(JobId=data_ingestion_job_id)

{'JobId': 'dfd3b0718667480afda43f8341b5eaec',
 'DatasetArn': 'arn:aws:lookoutequipment:us-east-1:593512547852:dataset/27b53ba8-baa0-4298-809a-0afe9384c280/3359ff20-cc69-484d-902a-cc09577d6eff',
 'IngestionInputConfiguration': {'S3InputConfiguration': {'Bucket': 'l4esitewisejjhj-asset1-train-inference',
   'Prefix': '27b53ba8-baa0-4298-809a-0afe9384c280/training-data/'}},
 'RoleArn': 'arn:aws:iam::593512547852:role/LookoutForEquipmentSitewisePipeli-SageMakerIamRole-1MJQSDGDJXJTP',
 'CreatedAt': datetime.datetime(2021, 9, 3, 2, 42, 54, 278000, tzinfo=tzlocal()),
 'Status': 'SUCCESS',
 'ResponseMetadata': {'RequestId': '3dea73af-cc01-41ab-8153-d4bb64a7326f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3dea73af-cc01-41ab-8153-d4bb64a7326f',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '515',
   'date': 'Fri, 03 Sep 2021 02:49:08 GMT'},
  'RetryAttempts': 0}}

The ingestion should now be complete as can be seen in the console:

![Ingestion done](assets/dataset-ingestion-done.png)

## Conclusion
---

In [10]:
# Needed for visualizing markdowns programatically
from IPython.display import display, Markdown

display(Markdown(
'''
<span style="color:green"><span style="font-size:50px">**Success!**</span></span>
<br/>
In this notebook, we created a **Lookout for Equipment dataset** and ingested the S3 data previously uploaded into this dataset. **Move now to the next notebook to train a model based on these data.**
'''))


<span style="color:green"><span style="font-size:50px">**Success!**</span></span>
<br/>
In this notebook, we created a **Lookout for Equipment dataset** and ingested the S3 data previously uploaded into this dataset. **Move now to the next notebook to train a model based on these data.**
